In [1]:
import pandas as pd
import random as rd
from collections import Counter
import time
import numpy as np

In [2]:
df = pd.read_csv('male_applicants.csv')

In [3]:
df

,names,category,weight
0,rqtttt,Ot,0.5
1,zhuztd,Ot,0.5
2,hqxfqg,Ot,0.5
3,sanjtt,Fy,0.5
4,mjbgpk,Ot,0.5
...,...,...,...
7361,bpgonl,Fr,0.5
7362,cvflwj,Fy,0.5
7363,carssw,Ot,0.5
7364,vtofrz,St,1.0


In [4]:
def generate_random_solution():
    category = df.category.values.tolist()
    category_copy = category.copy()
    hard_constrait = []
    for cand in category_copy:
        if cand == 'Ht' or cand == 'St':
            hard_constrait.append(cand)
            category.remove(cand)

    empty_space = 2072 - len(hard_constrait)

    soft_constrait = []
    for cand in range(empty_space):
        soft_constrait.append(rd.choice(category))

    solution = hard_constrait + soft_constrait
    
    return solution, hard_constrait, category

In [5]:
solution, _ , _= generate_random_solution()
frequency = Counter(solution)

show = pd.DataFrame()
for i in frequency: show[i] = [frequency[i]]
show.head()

,St,Ht,Ot,Sp,Fr,Fy
0,260,60,932,66,429,325


In [6]:
data = {'Fy': 1360, 'Fr':1660, 'Ht':60, 'Sp':290, 'St':260, 'Ot':3736}

soft_constr_weight = 1368 + 1660 + 290 + 3736
weight = {'Fy': 1 - (1360/soft_constr_weight), 'Fr': 1-(1660/soft_constr_weight), 'Ht':1, 'Sp':1- (290/soft_constr_weight), 'St':1, 'Ot': 1- (3736/soft_constr_weight)}

In [7]:
print(weight)

{'Fy': 0.8072015877516303, 'Fr': 0.7646725262262546, 'Ht': 1, 'Sp': 0.9588885738588036, 'St': 1, 'Ot': 0.47037142047065494}


In [8]:
def evaluate_solution(solution, weight):
    frequency = Counter(solution)
    loss = 0
    for category in solution: loss += weight[category] * frequency[category]
    return loss

In [9]:
def mutate_solution(hard_constrait, category):
#   mutate the solution by generating new soft constrait solution
    empty_space = 2072 - len(hard_constrait)

    soft_constrait = []
    for cand in range(empty_space):
        soft_constrait.append(rd.choice(category))

    solution = hard_constrait + soft_constrait
    
    return solution

In [10]:
def hill_climbing_CA(num_itr = 200):
    start_time = time.time()
    solution, hard_sol, soft_cat = generate_random_solution()
    cost = evaluate_solution(solution, weight)
    
    best_solution = solution
    cost_best_solution = cost
    print("Iteration 0 ------ cost:- {}".format(cost))
    
    for itr in range(1, num_itr+1):
        solution = mutate_solution(hard_sol, soft_cat)
        cost = evaluate_solution(solution, weight)
        
        if cost > cost_best_solution:
            print("Iteration {} ------ Found a better solution with cost:- {}".format(itr, cost))
            best_solution = solution
            cost_best_solution = cost
            
    end_time = time.time()
    
    print("\n Time taken - {}seconds".format(end_time-start_time))
    return best_solution, cost_best_solution

In [51]:
best_solution_CA, _ = hill_climbing_CA(200)

frequency = Counter(best_solution_CA)

best_solution_CA_dict = {i:j for i,j in frequency.items()}

show = pd.DataFrame()
for i in frequency: show[i] = [frequency[i]]
show.head()

Iteration 0 ------ cost:- 691977.9305358803
Iteration 1 ------ Found a better solution with cost:- 704052.2611284241
Iteration 2 ------ Found a better solution with cost:- 707893.3155656357
Iteration 3 ------ Found a better solution with cost:- 712020.2458179671
Iteration 4 ------ Found a better solution with cost:- 723481.1644457132

 Time taken - 0.6377794742584229seconds


,St,Ht,Fy,Ot,Fr,Sp
0,260,60,308,974,405,65


In [54]:
cc = best_solution_CA.copy()
stop = False
generated = {i:[] for i in HA_data.keys()}
while stop == False:
    if len(cc) == 0: stop = True
    for hall in HA_data:
        if len(generated[hall]) < HA_data[hall]:
            choice = rd.choice(cc)
            generated[hall].append(choice)
            cc.remove(choice)

In [66]:
HA_data = {'Akindeko': 625, 'Abiola':356, 'Adeniyi':195, 'New Male Hostel':792, 'Bisi Balogun':104}
order = ['Fy','Fr','Ht','Sp','St','Ot']

def generate_random_solution_HA(CA, HA_stat):
#     rd.shuffle(CA)
#     generated = {}
#     CA_copy = CA.copy()
#     for hall in HA_stat:
#         candidates = []
#         for count in range(HA_stat[hall]):
#             student = rd.choice(CA_copy)
#             candidates.append(student)
#             CA_copy.remove(student)
#         generated[hall] = candidates

    CA_copy = CA.copy()
    stop = False
    generated = {i:[] for i in HA_stat.keys()}
    while stop == False:
        if len(CA_copy) == 0: stop = True
        for hall in HA_stat:
            if len(generated[hall]) < HA_stat[hall]:
                choice = rd.choice(CA_copy)
                generated[hall].append(choice)
                CA_copy.remove(choice)
                
    generated = list(generated.values())
    
    matrix_solution = []
    for hall in generated:
        solution = []
        
        frequency = Counter(hall) #count each category in each hall
        for category in order:
            solution.append(frequency[category]) #append count of each category in each hall
        
        matrix_solution.append(solution) # concatenate solutions of all categories in all halls together
    
#     transpose solution to allow rows represent each category and column represent each hall
    matrix_solution = np.array(matrix_solution).transpose()
    return matrix_solution

In [67]:
solution_HA = generate_random_solution_HA(best_solution_CA, HA_data)

show = pd.DataFrame(solution_HA,
                    columns = ['Akindeko', 'Abiola', 'Adeniyi', 'New Male Hostel', 'Bisi Balogun'],
                    index = ['Fy','Fr','Ht','Sp','St','Ot'])

show

,Akindeko,Abiola,Adeniyi,New Male Hostel,Bisi Balogun
Fy,95,51,28,120,14
Fr,123,74,42,144,22
Ht,22,8,4,22,4
Sp,19,10,6,26,4
St,84,38,25,100,13
Ot,282,175,90,380,47


In [68]:
def evaluate_solution_FA(solution, data):
    solution = {i:j for i,j in zip(data, solution)}
    cost = 0
    for cat in solution: #loop through each category 
        proportion = data[cat] - (max(solution[cat]) - min(solution[cat]))
        N = sum(solution[cat] > 0 ) #number of halls that have some students of i-th category
        cost += proportion * N
    return cost

In [69]:
def hill_climbing_HA(num_itr = 200):
    start_time = time.time()
    solution = generate_random_solution_HA(best_solution_CA, HA_data)
    cost = evaluate_solution_FA(solution, best_solution_CA_dict)
    
    best_solution = solution
    cost_best_solution = cost
    print("Iteration 0 ------ cost:- {}".format(cost))
    
    for itr in range(1, num_itr+ 1):
        solution = generate_random_solution_HA(best_solution_CA, HA_data)
        cost = evaluate_solution_FA(solution, data)
        
        if cost > cost_best_solution:
            print("Iteration {} ------ Found a better solution with cost:- {}".format(itr, cost))
            best_solution = solution
            cost_best_solution = cost
            
    end_time = time.time()
    
    print("\n Time taken - {}seconds".format(end_time-start_time))
    return best_solution

In [71]:
show = pd.DataFrame(hill_climbing_HA(1000),
                    columns = ['Akindeko', 'Abiola', 'Adeniyi', 'New Male Hostel', 'Bisi Balogun'],
                    index = ['Fy','Fr','Ht','Sp','St','Ot'])

show

Iteration 0 ------ cost:- 6920
Iteration 1 ------ Found a better solution with cost:- 33343
Iteration 2 ------ Found a better solution with cost:- 33345
Iteration 3 ------ Found a better solution with cost:- 33390

 Time taken - 5.476413011550903seconds


,Akindeko,Abiola,Adeniyi,New Male Hostel,Bisi Balogun
Fy,91,50,35,117,15
Fr,120,58,39,171,17
Ht,19,11,7,20,3
Sp,16,9,6,32,2
St,78,45,27,99,11
Ot,301,183,81,353,56
